In [41]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import numpy as np
import torch
import torch.nn as nn

np.random.seed(42)
torch.manual_seed(42)    

# here we assume that p1 is already unsqueezed to match the p2 dimensions for broadcasting
# p1 = np.random.rand(5, 1000, 1, 3).astype('float32')

# p1 is unsqueezed to match the p2 dimensions for broadcasting
p1 = np.random.rand(5, 1000, 3).astype('float32')
p2 = np.random.rand(5, 1000, 16, 3).astype('float32')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
p1 = torch.tensor(p1).to(device)
p2 = torch.tensor(p2).to(device)

print (torch.all(p1[:, :, None, :] - p2 == p1.unsqueeze(-2) - p2))
p1 = p1.unsqueeze(-2)


tensor(True)


In [70]:
# # some numpy sidequest when I had an epiphany about broadcasting

# p1 = np.random.rand(5, 1000, 3).astype('float32')
# p2 = np.random.rand(5, 1000, 16, 3).astype('float32')

# sub_2 = p1[:, :, None, :] - p2
# sub_1 = np.tile(p1, (1,1,16)).reshape(5, 1000, 16, 3) - p2
# np.all(sub_1 == sub_2)

True

In [43]:
print (p1)
print (p2) 

tensor([[[[0.3745, 0.9507, 0.7320]],

         [[0.5987, 0.1560, 0.1560]],

         [[0.0581, 0.8662, 0.6011]],

         ...,

         [[0.8000, 0.5527, 0.3966]],

         [[0.1317, 0.8653, 0.1573]],

         [[0.3098, 0.2900, 0.8714]]],


        [[[0.6727, 0.7967, 0.2505]],

         [[0.6249, 0.5717, 0.8328]],

         [[0.9061, 0.0122, 0.6740]],

         ...,

         [[0.5420, 0.4596, 0.4990]],

         [[0.0722, 0.6844, 0.5032]],

         [[0.7651, 0.4853, 0.1494]]],


        [[[0.6483, 0.1724, 0.8724]],

         [[0.6131, 0.1572, 0.9623]],

         [[0.5184, 0.0729, 0.6268]],

         ...,

         [[0.2037, 0.3404, 0.4766]],

         [[0.2961, 0.5149, 0.1654]],

         [[0.9777, 0.1079, 0.9319]]],


        [[[0.9136, 0.5254, 0.7249]],

         [[0.4360, 0.6300, 0.3406]],

         [[0.3012, 0.5782, 0.4981]],

         ...,

         [[0.1556, 0.9600, 0.0378]],

         [[0.9776, 0.0046, 0.8737]],

         [[0.3073, 0.9386, 0.3169]]],


        [[[0.6543, 0

In [44]:
class PositionalEncoding(nn.Module):

    def __init__(self, out_dim):
        super().__init__()
        # since we take in xyz, out in_dim is always three, 
        # but out_dim needs to be whatever will fight later,
        # for transformer layer... right?
        self.linear_1 = nn.Linear(3, out_dim)
        self.linear_2 = nn.Linear(out_dim, out_dim)
        self.relu = nn.ReLU()
    
    def forward(self, p_i, p_j):
        diff = p_i - p_j
        out = self.linear_1(diff)
        out = self.linear_2(out)
        out = self.relu(out)
        return out

In [46]:
PositionalEncoding(3)(p1, p2).shape

torch.Size([5, 1000, 16, 3])